In [1]:
import idx2numpy
import numpy as np

from logistic_perceptron import LogisticPerceptron
from bagging import Bagging

In [2]:
def load_data(train_path_images, train_path_labels, test_path_images, test_path_labels):
    X_train = idx2numpy.convert_from_file(train_path_images)
    Y_train = idx2numpy.convert_from_file(train_path_labels)

    X_test = idx2numpy.convert_from_file(test_path_images)
    Y_test = idx2numpy.convert_from_file(test_path_labels)

    # Convertendo os rótulos para inteiros
    y_train = Y_train.astype(int)
    y_test = Y_test.astype(int)
    
    # vetorizando os dados
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    
    # normalização dos dados
    X_train = X_train / 255.0
    X_test = X_test / 255.0
    
    return X_train, y_train, X_test, y_test

In [3]:
# Carregando a base MNIST
train_path_images = 'dataset/train/train-images.idx3-ubyte'
train_path_labels = 'dataset/train/train-labels.idx1-ubyte'

test_path_images = 'dataset/test/t10k-images.idx3-ubyte'
test_path_labels = 'dataset/test/t10k-labels.idx1-ubyte'

X_train, Y_train, X_test, Y_test = load_data(train_path_images, train_path_labels, test_path_images, test_path_labels)

print(f'Dados de treino: \n{X_train} \nResultados de treino: {Y_train} \nDados de teste: \n{X_test} \nResultados de teste: {Y_test}')

Dados de treino: 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
Resultados de treino: [5 0 4 ... 5 6 8] 
Dados de teste: 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
Resultados de teste: [7 2 1 ... 4 5 6]


In [4]:
# Inicialização da classe que irá diversificar os dados
bagging = Bagging()

In [5]:
# Treinamento do comitê de perceptrons

num_committee = 5
committee_predicts = list()
predictions_matrix = list()

for i in range(num_committee):
    # Aplicando o bagging para embaralhar o dataset de treino com reposição
    sample, sample_labels = bagging.subsample(X_train, Y_train)
    print(f'------------------------------- Modelo {i+1} -------------------------------')
    
    # carregando o modelo
    model = LogisticPerceptron(
        X_train=sample, 
        Y_train=sample_labels, 
        X_val=X_test, 
        Y_val=Y_test, 
        epochs=100, 
        verbose=True,
    )
    
    # treinando o modelo
    model.fit()

    # armazenando os valores de inferencia dos modelos a cada n (valor padrao é 50 epocas) epocas em cima dos dados de teste
    predictions_matrix.append(model.predictions_matrix[0])
    

------------------------------- Modelo 1 -------------------------------
[TREINAMENTO] - Epoca 50 - Loss: 0.033580160164804376 - Predições: [7 2 1 ... 4 5 6] - Acurácia: 0.8348 - Taxa de Erro - 0.1652 2023-09-05 16:19:12.584848
[TREINAMENTO] - Epoca 100 - Loss: 0.04235587629761549 - Predições: [7 2 1 ... 4 8 6] - Acurácia: 0.7973 - Taxa de Erro - 0.2027 2023-09-05 16:19:29.667338
[TREINAMENTO] - Melhor MSE Loss: 0.018644049961175073 - Melhor acurácia: 0.9041 - 2023-09-05 16:19:29.687213
------------------------------- Modelo 2 -------------------------------
[TREINAMENTO] - Epoca 50 - Loss: 0.055302611163326326 - Predições: [7 2 1 ... 4 5 6] - Acurácia: 0.7782 - Taxa de Erro - 0.2218 2023-09-05 16:19:45.689991
[TREINAMENTO] - Epoca 100 - Loss: 0.018462168465275323 - Predições: [7 2 1 ... 4 5 6] - Acurácia: 0.9116 - Taxa de Erro - 0.0884 2023-09-05 16:20:02.775614
[TREINAMENTO] - Melhor MSE Loss: 0.018247926345864956 - Melhor acurácia: 0.9074 - 2023-09-05 16:20:02.809323
---------------

In [6]:
# Acuracia do comitê

# montando a matriz com todos os valores preditos por cada um dos modelos ao longo das epocas de treinamento
committee_predicts_ = np.stack(predictions_matrix, axis=1)

# # Aplicar votação majoritária em cada coluna da matriz committee_predicts_
final_predictions_by_epoch_ = np.apply_along_axis(bagging.majority_vote, axis=1, arr=committee_predicts_)

# Calcular a acurácia para o vetor com as predições mais votadas dentre todos os modelos
accuracy_ = np.mean(final_predictions_by_epoch_ == Y_test)
print('-----------------------------------------------------------------------------------------------------------------')
print(f'[INFERENCIA DO COMITÊ] Predições (voto majoritario): {final_predictions_by_epoch_} - Acurácia do Comitê na melhor época: {accuracy_:.4f}')
print('-----------------------------------------------------------------------------------------------------------------')
# -------------------------------------------------------------------------------------- #

-----------------------------------------------------------------------------------------------------------------
[INFERENCIA DO COMITÊ] Predições (voto majoritario): [7 2 1 ... 4 5 6] - Acurácia do Comitê na melhor época: 0.9096
-----------------------------------------------------------------------------------------------------------------
